In [30]:
import numpy as np
import pandas as pd
import datetime as dt
from app import db, WalliStat
import plotly.graph_objs as go       
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark" # available templates "plotly_dark" "plotly_white" 

In [33]:
def calc_walli_stats(year):
    UNUSED = ['I_L1', 'I_L2', 'I_L3', 'V_L1', 'V_L2', 'V_L3', 'extern_lock_state', 'charging_state', 
            'energy_pwr_on', 'I_max_cfg', 'I_min_cfg', 'modbus_watchdog_timeout', 'power_kW', 
            'remote_lock', 'I_max_cmd', 'I_fail_safe', 'campaign_id']
    ws_list = db.session.query(WalliStat).filter(WalliStat.campaign_id==0,
                                            WalliStat.datetime>=dt.date(year,1,1),
                                            WalliStat.datetime< dt.date(year,12,31)).all()
    df = WalliStat.to_dataframe(ws_list).drop(UNUSED, axis=1).set_index("datetime") 
    df["charged_kWh"] = df["energy_kWh"].diff()
    df["date"] = [idx.date() for idx in df.index]
    df["weekday"] = [idx.day_of_week for idx in df.index]
    df["week"] = [idx.weekofyear for idx in df.index]

    zeros = pd.DataFrame(columns=np.arange(1, 53, dtype=int), data=np.zeros(shape=(7, 52)))
    wks = df.pivot_table(index="weekday", columns="week", values="charged_kWh", aggfunc="sum") + zeros

    temps = df[["temperature", "date"]].groupby(by="date").agg(["max", "mean", "min"])
    temps.columns = [f"temperature {c}" for c in temps.columns.droplevel()]

    kwh = df[["date", "charged_kWh"]].groupby("date").agg("sum")
    kwh["rolling_mean"] = kwh["charged_kWh"].rolling(10, win_type="triang", center=True).mean()
    kwh["mean"] = [kwh.charged_kWh.mean()] * kwh.shape[0]
    
    return kwh, wks, temps

In [34]:
kwh, wks, temps = calc_walli_stats(dt.date.today().year)

In [35]:
fig = px.imshow(wks, labels={"x": "calender week", "y": "", "color": "charged_kWh"},
                color_continuous_scale='Greens', width=1000, height=300)
fig.update_layout(yaxis = {"tickmode": 'array',
                           "tickvals": [ 0,    1,    2,    3,    4,    5,    6  ],
                           "ticktext": ['Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa', 'Su']})
fig.show()

In [36]:
fig = go.Figure()
for col in temps.columns:
    fig.add_trace(go.Scatter(x=temps.index, y=temps[col], name=col))
fig.update_layout(width=1000, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

In [37]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=kwh.index, y=kwh.charged_kWh, mode="markers", name="charged_kWh"))
fig.add_trace(go.Scatter(x=kwh.index, y=kwh.rolling_mean, name="rolling_mean"))
fig.add_trace(go.Scatter(x=kwh.index, y=kwh["mean"], name="mean"))
fig.update_layout(width=1000, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show();